In [2]:
import numpy as np

<font color=darkblue>

# Risk in Crop Choice

In the crop choice model planners must choose between crops with different risk profiles. A discount rate is used to account for differences in these choices (i.e. invesments). This notebook describes thoughts on the use of discount rates in the crop choice model.

In the text below, *with some caveots discussed below* the term "investor" is interchangable with "planner" or "decision maker". The term "investment" is used to describe the choice between various crops.


## Discount Rate

Investors, are risk adverse. When given a choice between two investments with the same expected value $E[x]$, they will select the one with a less risk. Risk can be described by variability $var[x]$ in the range of possible outcomes. 

The discount rate for an investment $r$ is the rate of return that is required to compensate the investor for their patience, and the riskiness of the investment.

In practice, selecting a discount rate is an imprecise science. If we could measure investor patience and the risk premuim they demand as compensation for potential losses, then the present value, $PV$ of expected cash flows would be know and $r$ could be computed with the following equation:

>$r = \frac{FV}{PV}^{(1/n)} - 1$

where $FV$ is the future value of the investment cash flows (this is the nominal cash flow), and $n$ is the number of periods times the compounding frequency. In reality the discount rate is typically used to convert future cash flows into present values.


### Emperical Models

In some cases the discount rate can inferred from observation. For instance, companies issuing bonds for specific investments are able to observe, the cost of debt capital (a discount rate) based on the price the bonds fetch in the market place (ignoring tax benefits associated with debt).

The capital asset pricing model (CAPM) is used to make emperical inferences, usually about the cost of equity capital:

>$R_a = R_f + \beta(R_m - R_f)$

where $R_m$ is the expected return in the "market" (usually a boad index of stocks), $R_f$ is the expected "risk-free" return, and $R_a$ is the expected return from investment $a$. To identify a plausible discount rate, $r_a$ this equation is used as a regression based model, where a timeseries of observed returns for asset $a$ are regressed against excess market returns, $R_m - R_f$. 

If the investment is a "sure thing" it will deliver the stated return with certainty. In this case the the discount rate is the "risk-free" rate, $r_f$. In practice, US treasury bonds with a maturity matching $n$ (from the first equation) are assumed to be risk-free and applied in these situations. If the returns associated with treasuries are put on the left hand side of a CAPM regression model, $\beta$ a measure of the investment's riskiness will generally = 0, thus in this case $R_a = R_f$.

This regression based use of the CAPM model captures correlation between past returns of asset $a$ and excess returns ($R_m - R_f$). It cannot be used as a predictive model, especially if conditions affecting the expected risk premium or asset $a$ change. In the case of the crop choice model, changes in climate, surface and groundwater availability would be examples of shifts that would disrupt the correlation between crop returns and market returns. 

Another key assumption here is an efficient market, such that cost of assets which continously perform above their required rate would be bid until the $r_a \approx R_a$.



### Systematic and Idiosyncratic Risk

The discount rate $r$, is composed of: (a) the risk-free rate $r_f$, which captures the pure time preference for money, and (b) a risk premium, that compensates investors for downside risks.

The CAPM model $\beta$ measures a difference between the average risk premium $\beta = 1$ (in the market as a whole) and the risk premium demanded for investment in asset $a$. If $\beta < 1$ then the investment is less risky than the market as a whole, while investments with $\beta > 1$ are more risky (and therefore require a higher expected return) than the market as a whole. Therefore, $\beta$ measures the relative or idiosyncratic risk of the investment.

This is usaully important as a part of portfolio theory, where investors reduce the overall riskiness of their investments though diversification - such that only systematic risk remains and the co-variance between a new investment and the portfolio is of particular importance.

### Sharpe Ratio

Another equation that is used to model the systematic and idiosyncratic investment risk is the sharpe ratio:
> $s=\frac{r_m - r_f}{\sigma_m}$
where $\sigma_m$ is the standard deviation of returns in the market.

In general, a higher sharpe ratio, $s$ is desirable since it indicates a higher expected return $r_m - r_f$ - per unit of risk. The equation can be rearranged to express the market return as function of sharpe ratio.
> $r_m = s \times \sigma_m + r_f$

This can be used to estimate a required rate of return for asset $a$, given an overall level of idiosyncratic and systematic risk.
> $r_a = s \times \sigma_a \times \gamma + r_f$

where $\gamma$ is systematic risk, e.g. the portion of volitility in returns from asset $a$ after diversification. 

In important sharpe ration limitation is that risk, measured by $\sigma_a$ is assumed to be symetrical (and not excessively kurtotic). I think this limitation matters more in terms of the sharpe ratio's *normative* use (a preference for higher sharpe ratios), but it is an **exceptionally bad assumption for the crop choice model** since production risks as they relate to water resources are skewed almost entirely towared negative outcomes (and probably exhibit excess kurtosis) in a managed system.

An attractive aspect of the formulation of the sharpe ratio above, for the crop model is that $\gamma$ is non-dimensional variable that adjusts the variance in the returns. 

Holding $\gamma$ constant increasing $\sigma$ increases the required rate of return. In reality, some increases in variance (and therefore $\sigma$) may be diversified away (decreasing $\gamma$), but in an experimental setting given a $r_m, r_f,$ and $\gamma$ this formulation provides a method of adjusting $r_a$ in response to an exogenous shift in $\sigma_a$.

<font color=red>

**If I've stated anything new above it is a mistake ~ upto this point I've only been framing my thoughts**

## Discount rates in the crop choice model.

The main idea surrounding the use of discount rates in crop choice is to model the impact of that static and evolving climate based risks have on decisions. Two possible frameworks can be explored:

### Discounting the past

Planners may decide to discount the information provided in older observations, instead building expectations about the future from increasing myopic records. The model facilitates this with the **expected_value()** function in the *utilities* module, described in the *utilities.ipynb* notebook. 

If timeseries used to build these expectations, are not augmented in some way it is unclear if this will result in a increased or descreased preception of risk - only that older deviation from the mean would be de-valued in favor of more recent deviations from the mean.

### Making forecasts

Alternatively, planners may choose to make forecast about the future - using past trends or information provided from an ensemble of GCMs. 

1. natural variability on profit - this requires a distribution within climate state values (e.g. a distribution of ETo, kc, precipitation and surfacewater availability).

2. climate change on profit - this requires a distribution of possible climate states (e.g. a distribution of ETo, kc, precipitation, and surface water availability) by year.

______

In the crop model baseline risk, $\sigma_a$ is defined by natural variability in the climate (which impacts: ETo, Kc, Preciptitation, and Surfacewater availability). Letting $\alpha$ stand for annual crops, and $\phi$ stand for perennial (tree) crops in the baseline case $\sigma_\alpha$ and $\sigma_\phi$ need not be equal.

Changes in climate, can impact the discount rate though two seperate mechanisms:

1. For an investment of $n$ years, the risk at any given time period $t$: $\sigma_{a,t}$ will be based on natural variability in the observational record. Note that **d**: the rate at which past observations are incorporated into expectations about the future (**see expected value function component of the utilities.ipynb**) may be used to attribute fewer past observations to this natural variability.

2. 

1. risks associated with annual crops will change slowly, meaning $\sigma_{\alpha,t-1} \approx \sigma_{\alpha,t}$ since the investment is for a single year (variability will be dominated by natural variability). The portion 

2. $\sigma_{\phi, t} > \sigma_{\alpha, t} since the perennial crop is exposed to risks associated with natural variability and climate change. The magnitude of this increase in risk could be measured by